# Data Collection

In order to avoid collision with obsticals, data must be collected to train the robot on when the path in front of it is ``free`` or ``blocked``.  This notebook connects a gamepad controller and save camera images for the dataset.

### Gamepad Controller
1. Visit [http://html5gamepad.com](http://html5gamepad.com).  
2. Press buttons on the gamepad you're using
3. Remember the ``index`` of the gamepad that is responding to the button presses

In [ ]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # replace with index of your controller from web site
display(controller)

Press a button and you should see the gamepad widget appear above.

### Robot Motors
Instantiate the robot object to access the motors/servos.

In [ ]:
from jetbot import Robot
import traitlets

robot = Robot()

### Link Gamepad to Wheels

In [ ]:
left_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.axes[3], 'value'), (robot.right_motor, 'value'), transform=lambda x: -x)

The *left* stick controls the *left* motor and the *right* stick controls the *right* motor.

### Camera
A widget is needed for the user to see the what the robot sees.

In [ ]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg

camera = Camera.instance()
image = widgets.Image(format='jpeg', width=300, height=300)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

### Dataset
Create dataset folders for training.

In [ ]:
import os

blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

### Snapshot
Save snapshots classified as ``free`` or ``blocked`` into their respective folders.

In [ ]:
from uuid import uuid1

def save_snapshot(directory):
    # save picture with unique file name
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free(change):
    # save new image as free
    if change['new']:
        global free_dir, free_count
        save_snapshot(free_dir)
        free_count.value = len(os.listdir(free_dir))
    
def save_blocked(change):
    # save new image as blocked
    if change['new']:
        global blocked_dir, blocked_count
        save_snapshot(blocked_dir)
        blocked_count.value = len(os.listdir(blocked_dir))

In [ ]:
# display number of each type of picture
layout = widgets.Layout(width='256px', height='32px')
style = {'description_width': 'initial'}
free_count = widgets.IntText(description='Free Count', layout=layout, style=style, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(description='Blocked Count', layout=layout, style=style, value=len(os.listdir(blocked_dir)))
display(widgets.HBox([free_count]))
display(widgets.HBox([blocked_count]))

In [ ]:
# link the bumper buttons to the save action
controller.buttons[4].observe(save_free, names='value')
controller.buttons[5].observe(save_blocked, names='value')

### Cleanup
Stop the camera before closing notebook so that others can access it.

In [ ]:
camera.stop()